# Large Eddy Simulator for Urban Flows

The code in `src` in this repository has support for MPI and Nested Grids. OpenCL code for GPU and CPU can be generated from this code as well.

For more details see [Nested-LES.ipynb](http://localhost:7777/notebooks/Nested-LES.ipynb)

2018-09-13

Aims: 

- Validate MPI CPU code without nesting, against non-MPI CPU code, specifically to test
- Validate MPI CPU code without nesting, specifically to test
    MPI_NEW_WV 
    WV_NEW
    MPI_NEW_WV and WV_NEW
    
- Validate GPU code against MPI CPU code without nesting.

## Validation

Short run (100 time steps, then 500), compare uspd/vspd and pressure/vel at centre

$$    
uspd_{i,j}=\sqrt{u_{i,j,1}^{2}+\frac{1}{2}\left(\frac{(v_{i,j-1,1}+v_{i,j,1})\Delta x_{i+1}+(v_{i+1,j-1,1}+v_{i+1,j,1})\Delta x{}_{i}}{\Delta x{}_{i}+\Delta x{}_{i+1}}\right)^{2}}
$$

And similar for _vspd_. The number of time steps is of course too small: the LES outputs only one point every 1000 steps.

Script:

    ./build_and_run_WV_MPI_no_nesting_DPRI.sh 2 2

            

    

### STATUS 

- The code without WV_NEW or MPI_NEW_WV now compiles and runs
- There seems to be no difference between baseline and MPI_NEW_WV, but I observe that 2 runs do not produce the same pressures

### Memory footprint reduction

- After splitting out `WV_NEW` functionality with additional `WV_NEW_FEEDBF`,`WV_NEW_LES`,`WV_NEW_LES2`,`WV_NEW_VEL2` and `WV_NEW_VELFG`, the new functionality works. The combined effect is to eliminate all `*mask`, `cov*`, `nou*` and `diu*` arrays. The full picture:

* Before: 73 3-D arrays, 16 for aveflow

        #ifdef I_AVEFLOW
            real(kind=4), dimension(ip,jp,kp)  :: avel
            real(kind=4), dimension(ip,jp,kp)  :: avep
            real(kind=4), dimension(ip,jp,kp)  :: avesm
            real(kind=4), dimension(ip,jp,kp)  :: avesmsm
            real(kind=4), dimension(ip,kp)  :: avesu
            real(kind=4), dimension(ip,kp)  :: avesuu
            real(kind=4), dimension(ip,kp)  :: avesv
            real(kind=4), dimension(ip,kp)  :: avesvv
            real(kind=4), dimension(ip,kp)  :: avesw
            real(kind=4), dimension(ip,kp)  :: avesww
            real(kind=4), dimension(ip,jp,0:kp)  :: aveu
            real(kind=4), dimension(ip,jp,kp)  :: aveuu
            real(kind=4), dimension(ip,jp,0:kp)  :: avev
            real(kind=4), dimension(ip,jp,kp)  :: avevv
            real(kind=4), dimension(ip+1,jp,0:kp+2)  :: avew
            real(kind=4), dimension(ip,jp,kp)  :: aveww
        #endif    
            real(kind=4), dimension(0:ip+1,0:jp+1,0:kp+1)  :: amask1
            real(kind=4), dimension(-1:ip+1,0:jp+1,0:kp+1)  :: bmask1
            real(kind=4), dimension(0:ip+1,-1:jp+1,0:kp+1)  :: cmask1
            real(kind=4), dimension(0:ip+1,0:jp+1,0:kp+1)  :: dmask1
            real(kind=4), dimension(ip,jp,kp)  :: cn1
            real(kind=4), dimension(-1:ip+2,0:jp+2,0:kp+2)  :: cov1
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: cov2
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: cov3
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: cov4
            real(kind=4), dimension(-1:ip+2,0:jp+2,0:kp+2)  :: cov5
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: cov6
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: cov7
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: cov8
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: cov9
            real(kind=4), dimension(0:ip,jp,kp)  :: dfu1
            real(kind=4), dimension(ip,0:jp,kp)  :: dfv1
            real(kind=4), dimension(ip,jp,kp)  :: dfw1
            real(kind=4), dimension(-1:ip+2,0:jp+2,0:kp+2)  :: diu1
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: diu2
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: diu3
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: diu4
            real(kind=4), dimension(-1:ip+2,0:jp+2,0:kp+2)  :: diu5
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: diu6
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: diu7
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: diu8
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: diu9
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: f
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: g
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: h

            real(kind=4), dimension(ip,jp,kp)  :: fold
            real(kind=4), dimension(ip,jp,kp)  :: gold
            real(kind=4), dimension(ip,jp,kp)  :: hold
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: fx
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: fy
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: fz
            real(kind=4), dimension(-1:ip+2,0:jp+2,0:kp+2)  :: nou1
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: nou2
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: nou3
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: nou4
            real(kind=4), dimension(-1:ip+2,0:jp+2,0:kp+2)  :: nou5
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: nou6
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: nou7
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: nou8
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+2)  :: nou9
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+1)  :: p
            real(kind=4), dimension(0:ip+1,0:jp+1,0:kp+1)  :: rhs
            real(kind=4), dimension(-1:ip+1,-1:jp+1,0:kp+1)  :: sm
            real(kind=4), dimension(0:ip+1,-1:jp+1,0:kp+1)  :: u
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: usum
            real(kind=4), dimension(ip,jp,kp)  :: uwfx
            real(kind=4), dimension(0:ip+1,-1:jp+1,0:kp+1)  :: v
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: vsum
            real(kind=4), dimension(0:ip+1,-1:jp+1,-1:kp+1)  :: w
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: wsum
            real(kind=4), dimension(-1:ipmax+1,-1:jpmax+1)  :: zbm
            real(kind=4), dimension(0:ip+1,0:jp+1)  :: uspd
            real(kind=4), dimension(0:ip+1,0:jp+1)  :: vspd

* After: 26 3-D arrays, 9 for aveflow

        #ifdef I_AVEFLOW
            real(kind=4), dimension(ip,jp,kp)  :: avep
            real(kind=4), dimension(ip,jp,kp)  :: avesm
            real(kind=4), dimension(ip,jp,kp)  :: avesmsm
            real(kind=4), dimension(ip,jp,0:kp)  :: aveu
            real(kind=4), dimension(ip,jp,kp)  :: aveuu
            real(kind=4), dimension(ip,jp,0:kp)  :: avev
            real(kind=4), dimension(ip,jp,kp)  :: avevv
            real(kind=4), dimension(ip+1,jp,0:kp+2)  :: avew
            real(kind=4), dimension(ip,jp,kp)  :: aveww
        #endif    

            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: f

            real(kind=4), dimension(ip,jp,kp)  :: fold
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: g
            real(kind=4), dimension(ip,jp,kp)  :: gold
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: h
            real(kind=4), dimension(ip,jp,kp)  :: hold
        #ifndef TWINNED_BUFFER
            real(kind=4), dimension(0:ip+2,0:jp+2,0:kp+1)  :: p
        #else
            real(kind=4), dimension(0:1,0:ip+2,0:jp+2,0:kp+1)  :: p
        #endif
            real(kind=4), dimension(0:ip+1,0:jp+1,0:kp+1)  :: rhs
            real(kind=4), dimension(-1:ip+1,-1:jp+1,0:kp+1)  :: sm
            real(kind=4), dimension(0:ip+1,-1:jp+1,0:kp+1)  :: u
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: usum
            real(kind=4), dimension(ip,jp,kp)  :: uwfx
            real(kind=4), dimension(0:ip+1,-1:jp+1,0:kp+1)  :: v
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: vsum
            real(kind=4), dimension(0:ip+1,-1:jp+1,-1:kp+1)  :: w
            real(kind=4), dimension(0:ip,0:jp,0:kp)  :: wsum
            real(kind=4), dimension(-1:ipmax+1,-1:jpmax+1)  :: zbm

* Testing the MPI baseline on the Mac with following settings:

        #define NO_GLOBAL_SOR 
        #define WV_TIMESTEPS 500
        #define NO_IO 
        #define IFBF 1 
        #define IADAM 0 
        #undef I_IFDATA_OUT
        #undef I_AVEFLOW
        #undef I_ANIME

This gives 375MB per MPI process for 4 processes and a 300x300x80 domain

* Testing the improved version on the Mac with following settings:

        #define MPI_NEW_WV
        #define WV_NEW_FEEDBF
        #define WV_NEW_LES
        #define WV_NEW_LES2
        #define WV_NEW_VEL2
        #define WV_NEW_VELFG
        #define WV_NEW
        #define NO_GLOBAL_SOR 
        #define WV_TIMESTEPS 500
        #define NO_IO 
        #define IFBF 1 
        #define IADAM 0 
        #undef I_IFDATA_OUT
        #undef I_AVEFLOW
        #undef I_ANIME

This gives 198MB per MPI process for 4 processes and a 300x300x80 domain, so an improvement of almost a factor of 2. Going purely on the 3-D arrays it should have been closer to 3.3x. Maybe there are some subroutine-local arrays I overlooked? There are 4 arrays in `anime` but that is unused with this setup.



I think there are still way too many arrays. I should check if I can get rid of the `ave*` ones. And what about `{u,v,w,}sum`?




- Nested LES compiles wiht these new features but is yet to be tested.

### WV_NEW Debugging 

#### (for historical info only)

- `WV_NEW` produces quite different results, but this might be a transient, I will have to run for longer.
- OK, did that and it NaNs. And I checked without MPI, and it stil NaNs. So there must be an actual mistake. Let's find out. 

  - The following files have WV_NEW:

        adam.f95
        anime.f95
        aveflow.f95
        bondFG.f95
        bondv1.f95
        boundp.f95
        boundsm.f95
        feedbf.f95
        feedbfm.f95
        grid.f95
        ifdata.f95
        init.f95
        les.f95
        main.f95
        press.f95
        set.f95
        timseris.f95
        vel2.f95
        velFG.f95
        velnw.f95

  - So that is all of them, really. The following are guaranteed OK:

        adam.f95
        anime.f95
        aveflow.f95
        bondFG.f95        
        bondv1.f95
        boundp.f95
        boundsm.f95
        grid.f95
        timseris.f95
        velnw.f95
        vel2.f95        
        
  - Most likely OK: (assuming that we did not eliminate too many arrays)

        feedbfm.f95
        ifdata.f95
        init.f95
        main.f95        
        set.f95 

  - Critical subroutines, most likely to contain the bug:

        feedbf.f95 => no MPI
        les.f95 => MPI only via call to boundsm
        press.f95      
        velFG.f95 => MPI only via call to bondfg       
        
    For `feedbf.f95` there is a macro `NOT_INLINED` which I can set to test.       
    In `press.f95`, the macro `WV_OPENCL` interacts with `WV_NEW`, so it is possible that is is only wrong without OpenCL.

- I also compared 'vanilla' MPI with 'vanilla' non-MPI and the difference in uspd/vspd is very considerable. But it does not NaN.
        


- See if I can isolate the `WV_NEW` functionality so that testing what is wrong is easier. 
- Start with `feedbf`, though I doubt that the problem is there. I replaced `WV_NEW` in feedbf with `WV_NEW_FEEDBF`. I will do the same for the changes in velFG, les and press. Unsurprisingly, this still NaNs, so it is not the root cause. 
- Let's try the same with `les`: `WV_NEW_LES`, again, still NaNs.
- Next, `velfg`, so we define `WV_NEW_VELFG`. This does not NaN but the result is completely wrong, so I must have made a mistake in the macros here.
- OK, hopefully fixed all that. And now the bug is gone, which means that the cause is in `velfg` I suppose
- So now I should be able to switch on `WV_NEW_FEEDBF` and `WV_NEW_LES` and still get the correct results.
- For `WV_NEW_FEEDBF` this is OK!
- For `WV_NEW_LES` this is not possible because `WV_NEW_LES` eliminates the `diu*` arrays used by `velfg`. So as a first step we need to adapt `velfg` to work without `diu*` only, keeping the other arrays. 
- I did that, removed arrays in `vel2`, as far as I can tell they are not used outside it, so no point in computing bounds on them. And yes, it works, although the result is a little bit different from the result with the arrays. To be sure, I ran a sim for 500 timesteps, and it's fine.
- Meanwhile, lets remove `nou*`, as this is not used outside `velfg` so also not outside `vel2` I guess. I did this using `WV_NEW_LES2`. I ran a sim for 500 timesteps, and it's fine.
- Finally, let's look at `cov*`, as the problem must be there (or in the inlined boundary calcs? unlikely!).
- I do this in two steps: 
 (1) merge the (2-D) boundary calculations into the 3-D loops, using `WV_NEW_VEL2`. This is fine (run of 100)
 (2) scalarise the loops to ultimately remove `cov*`, and that should be `WV_NEW_VELFG` I guess.
 I checked the code and found that I had omitted some boundary conditions (k=1) for cov3 and cov6. But it still NaNs, so back to the drawing board.
- OK, now it is fixed. Essentially there were errors in the boundary conditions. I've done a sim for 500 timesteps, and it's fine. Now doing an MPI run for 500 steps, then one for 10000 steps.
- However, running 10000 steps with MPI NaNs, after less than 1000 steps.
- However however, running 10000 steps without MPI does not NaN. Still, the values are not the same as for the MPI run. This is not so surprising as the point where the values are taken is a bit different.
- So now I must find out why the MPI version NaNs. 
    - I can see no reason in velFG itself.
    - feedbf(m) seems OK, all we did is localise the `*mask` arrays
    - It is possible that the problem is rather `MPI_NEW_WV` as I only ran this for 50 steps.
    - No, I checked running with `MPI_NEW_WV` undefined and it still NaNs
    - So I must go back and undef the WV_NEW features. Starting with `WV_NEW_VELFG`
    - The outcome is, sadly, that it works. So `WV_NEW_VELFG` must somehow be wrong, but only if we do MPI
    
        #define MPI_NEW_WV
        #undef NESTED_LES
        #define WV_NEW_FEEDBF
        #define WV_NEW_LES
        #define WV_NEW_LES2
        #undef WV_NEW_VELFG
        #define WV_NEW
        #define WV_TIMESTEPS 10000
        #define NO_GLOBAL_SOR 
        #define IFBF 1 
        #define IADAM 0 
        #undef I_IFDATA_OUT
        #undef I_AVEFLOW
        #define I_ANIME

    - Ah, I think I've got it: the boundary conditions! 
    
### Fixing the `WV_NEW_VELFG` case

```fortran
The original code was:

        #if !defined(MPI) || (PROC_PER_ROW==1)
              do k = 1,kp
              do i = 1,ip
                nou2(i,0,k) = nou2(i,jp,k)
                nou2(i,jp+1,k) = nou2(i,1,k)
              end do
              end do
        #else
            call sideflowRightLeft(nou2, procPerRow, jp+1, 1, 1, 2, 1, 2)
            call sideflowLeftRight(nou2, procPerRow, 2, jp+2, 1, 2, 1, 2)
        #endif
```

The current code is:
 
```fortran
          do k = 1,kp
              do j = 1,jp
                  do i = 1,ip
                      nou2_ = (dx1(i+1)*v(i,j-1,k)+dx1(i)*v(i+1,j-1,k)) /(dx1(i)+dx1(i+1))
                      diu2_ = 2.*(-u(i,j-1,k)+u(i,j,k))/(dy1(j-1)+dy1(j))
                      cov2_j = nou2_*diu2_
                      if (j<jp) then
                          nou2_jp1 = (dx1(i+1)*v(i,j,k)+dx1(i)*v(i+1,j,k)) /(dx1(i)+dx1(i+1))
                          diu2_jp1 = 2.*(-u(i,j,k)+u(i,j+1,k))/(dy1(j)+dy1(j+1))
                          cov2_jp1 = nou2_jp1*diu2_jp1
                      else
                          nou2_ = (dx1(i+1)*v(i,0,k)+dx1(i)*v(i+1,0,k)) /(dx1(i)+dx1(i+1))
                          diu2_ = 2.*(-u(i,0,k)+u(i,1,k))/(dy1(0)+dy1(1))
                          cov2_jp1 = nou2_*diu2_
                      end if 
                  end do          
              end do          
          end do                    
```          

This is wrong because if `j==jp` then we can't access the values at j=0 and j=1. So I should copy these values with MPI.

So the sender process should compute them: 

- If we are in the left sideplane, we calculate  

          nou2_1 = (dx1(i+1)*v(i,0,k)+dx1(i)*v(i+1,0,k)) /(dx1(i)+dx1(i+1))
          diu2_1 = 2.*(-u(i,0,k)+u(i,1,k))/(dy1(0)+dy1(1))
          cov2_1 = nou2_1*diu2_1  

and we sent this to the right sideplane. We can actually just compute `cov2_` and send it only if `j==1`

- If we are in the right sideplane, and `j==jp`, we block to receive `cov2_1`, then we say

          cov2_jp1 = cov2_1

- This is inefficient, what we should do of course is store this in an  array.

- The routine

        sideflowLeftRight(array, procPerRow, colToSend, colToRecv, &
                topThickness, bottomThickness, ignoreFirstK, ignoreLastK)

assumes that array is a 3-D array but we could define 

        real, dimension(ip,1,kp) :: cov2_left_plane 

and I think we call it like

        sideflowLeftRight(cov2_left_plane, procPerRow,1,1,0,0,0,0)
        
and then we should use these values to recompute. However, I think a better way is to just update the bounds on f,g and h. The values of u,v,w do not change after this call. So updating f,g,h the way we do should be fine. Fingers crossed.         
- Somehow this is wrong, it NaNs now very quickly was before. So just turning off these boundary calculations does not work. I switch the open ones (ip+1 = ip) on in MPI. 
- This is still not good, which means that my assumptions on the values in u,v,w must be incorrect.
- I rewrote some expressions which seem to have uspd/uspd in them, not sure this can make a difference as the compiler should do this, right? And indeed, it does not make a difference.
- So it boils down to this then:

        nou2_jp1 = (dx1(i+1)*v(i,jp,k)+dx1(i)*v(i+1,jp,k)) /(dx1(i)+dx1(i+1))
        diu2_jp1 = 2.*(-u(i,jp,k)+u(i,jp+1,k))/(dy1(jp)+dy1(jp+1))
        cov2_jp1 = nou2_jp1*diu2_jp1

        ! cov2(i,jp+1,k) = cov2(i,1,k)
        #if !defined(MPI) || (PROC_PER_ROW==1)
              if (j==jp) then
                  nou2_ = (dx1(i+1)*v(i,0,k)+dx1(i)*v(i+1,0,k)) /(dx1(i)+dx1(i+1))
                  diu2_ = 2.*(-u(i,0,k)+u(i,1,k))/(dy1(0)+dy1(1))
                  cov2_jp1 = nou2_*diu2_
              end if
        #endif

        nou5_jp1 = ( v(i,jp,k)+v(i,jp+1,k))/2.
        diu5_jp1 = (-v(i,jp,k)+v(i,jp+1,k))/dy1(jp+1)
        cov5_jp1 = nou5_jp1*diu5_jp1

        ! cov5(i,jp+1,k) = cov5(i,1,k)
        #if !defined(MPI) || (PROC_PER_ROW==1)        
              if (j==jp) then
                  nou5_ = ( v(i,1,k)+v(i,2,k))/2.
                  diu5_ = (-v(i,1,k)+v(i,2,k))/dy1(j)
                  cov5_jp1 = nou5_*diu5_
              end if  
        #endif

        nou8_jp1 = (dzn(k+1)*v(i,jp,k)+dzn(k)*v(i,jp,k+1)) /(dzn(k)+dzn(k+1))
        diu8_jp1 = 2.*(-w(i,jp,k)+w(i,jp+1,k))/(dy1(jp)+dy1(jp+1))
        cov8_jp1 = nou8_jp1*diu8_jp1
      
        !cov8(i,jp+1,k) = cov8(i,1,k)
        #if !defined(MPI) || (PROC_PER_ROW==1)
              if (j==jp) then
                nou8_ = (dzn(k+1)*v(i,0,k)+dzn(k)*v(i,0,k+1)) /(dzn(k)+dzn(k+1))
                diu8_ = 2.*(-w(i,0,k)+w(i,1,k))/(dy1(0)+dy1(1))
                cov8_jp1 = nou8_*diu8_
              end if
        #endif
        
So my debug strategy is:

1. Check if it NaNs without MPI if I turn off all boundary handling => Yes, it does!
2. If it does, check if it NaNs without MPI if I turn off only the circular boundary handling => No, it does not, at least not for the first 500 steps. So, what does this mean? That for the case without MPI, the circular boundary conditions do not matter. This is what I thought as bondv1 should actually take care of that. So this seems to indicate that there is a problem with the open condition in MPI!
3. So test MPI with proper open conditions, it looks like I mixed up Top and Bottom => Yes, this is fine, up to 4000 steps at least




## Scripts for building and generating input files

### SCons build script

In [38]:
import os
import os.path # for getting environmental variables on the system

# Importing OclBuilder, this is not required for ocl=0
# If you want to build just the Fortran code without OpenCL support, use SConstruct.F95_only
import OclBuilder
from OclBuilder import initOcl

# Adding path to includes for kernels
CWD= os.environ['PWD']
OclBuilder.kopts='-cl-mad-enable -cl-fast-relaxed-math -I'+CWD+'/../OpenCL/Kernels/'

from OclBuilder import getOpt
OclBuilder.opts=Variables()
envF=Environment(useF=1)
envF=Environment(ENV={'PATH' : os.environ['PATH']})


# Basically, it's Linux unless it's OS X
if os.uname()[0] == "Darwin":
        OSX=1
        OSFLAG='-DOSX'
else:
        OSX=0
        OSFLAG='-D__LINUX__'


# Then build the rest of the code
verbose = getOpt('v','Verbose','1')
other=''
WITH_OCL=''
with_ocl= getOpt('ocl','Use OpenCL','0')
if with_ocl == '1':
    WITH_OCL = '-D_OPENCL_LES_WV'
    envF=initOcl(envF)
    kernel_opts = envF['KERNEL_OPTS']
else:
    envF['F95']=os.environ['FC']
    envF['LINK']=os.environ['FC']
    VERBOSE = '-DVERBOSE'
    if verbose == '0':
        VERBOSE = ''
    other = getOpt('D','Other macro','')
    TIMINGS=''
    if other !='':
        TIMINGS = '-D'+other

# GR: MPI specific build config
WITH_MPI=''
PROC_PER_ROW=''
PROC_PER_COL=''
USE_NETCDF_OUTPUT=''
with_mpi = getOpt('mpi','Use MPI','0')
with_nesting = getOpt('nested','Nested Grid','0')
if with_mpi == '1':
    procPerRow= getOpt('procPerRow', 'Processes Per Row', '2')
    procPerCol= getOpt('procPerCol', 'Processes Per Col', '2')
    PROC_PER_ROW = '-DPROC_PER_ROW=' + procPerRow
    PROC_PER_COL = '-DPROC_PER_COL=' + procPerCol
    WITH_MPI = '-DMPI'
    USE_NETCDF_OUTPUT='-DUSE_NETCDF_OUTPUT'
#    envF['F95']='mpiifort'
#    envF['F95']='mpif90'
    if OSX==0:    
        envF['LINK']=envF['F95']
        envF.Append(LIBS=['netcdf']) # for version less than 4.2.0
    else:
        envF['F95']=os.environ['FC']
        envF['LINK']=os.environ['FC']
        envF.Append(LIBS=['mpi_mpifh','netcdff']) # for version more than and equal to 4.2.0 
else:
    if OSX==1:    
        envF['F95']=os.environ['FC']
        envF['LINK']=os.environ['FC']
        envF.Append(LIBS=['netcdff']) # for version more than and equal to 4.2.0 

NESTED_LES=''
if with_nesting=='1':
	NESTED_LES='-DNESTED_LES'
		
GR_DEBUG=''
gr_debug = getOpt('gr_debug', 'GR Debug', '0')
if gr_debug == '1':
    GR_DEBUG='-DGR_DEBUG'

WV_DEBUG=''
wv_debug = getOpt('wv_debug', 'WV Debug', '0')
if wv_debug =='1':
    WV_DEBUG='-DMPI_NEW_WV'

NO_FILE_IO='-DNO_FILE_IO'
ICAL = '-DICAL=0'
IFBF='-DIFBF=1'
IANIME='-DIANIME=1'
IADAM='-DIADAM=0'
FFLAGS  = [USE_NETCDF_OUTPUT, WITH_MPI, NESTED_LES, GR_DEBUG, WV_DEBUG, PROC_PER_ROW, PROC_PER_COL, WITH_OCL, NO_FILE_IO, ICAL, IFBF,IANIME, IADAM]

if with_ocl == '0':
#    FFLAGS += ['-cpp', '-O', '-Wall','-ffree-form', '-ffree-line-length-none','-fconvert=big-endian', '-mcmodel=medium', VERBOSE,TIMINGS]
#     FFLAGS += ['-cpp', '-O', '-Wall','-ffree-form', '-ffree-line-length-none','-fconvert=big-endian', '-mcmodel=medium', '-fno-range-check','-fbounds-check','-Wuninitialized','-ffpe-trap=invalid,zero,overflow', VERBOSE,TIMINGS]
     FFLAGS += ['-cpp', '-O', '-Wall','-ffree-form', '-ffree-line-length-none','-fconvert=big-endian', '-mcmodel=medium','-fbounds-check', VERBOSE,TIMINGS]
#    FFLAGS += ['-cpp','-Ofast', '-m64', '-Wall','-ffree-form', '-ffree-line-length-none','-fconvert=big-endian', VERBOSE,TIMINGS]

csources=[]
CC= os.environ['CC']
envC=Environment(CC=CC)
if csources:
    envC.Library('csubs',csources)

fsources = []

if with_mpi == '1':
    fsources += ['./communication_common.f95', './communication_helper.f95', './communication_helper_integer.f95', './communication_helper_mpi.f95', './communication_helper_real.f95']
    
if with_nesting == '1':
    fsources += ['./nesting_support.f95']

if USE_NETCDF_OUTPUT != '':
    fsources += ['./module_LES_write_netcdf.f95']

fsources+= ['./fortran_helper.f95', './anime.f95','./aveflow.f95','./bondFG.f95','./bondv1.f95','./boundp.f95','./boundsm.f95','./vel2.f95','./velFG.f95','./feedbf.f95','./feedbfm.f95','./les.f95','./grid.f95','./ifdata.f95','./init.f95','./main.f95','./set.f95','./timdata.f95','./common_sn.f95','./params_common_sn.f95']

ffsources=[]

if with_ocl == '1':
    ffsources = fsources + ['./module_LES_conversions.f95','./module_LES_combined_ocl.f95','./oclWrapper.o']
else:
    ffsources = fsources + ['./adam.f95','./press.f95','./velnw.f95']


if with_ocl == '1':
    # Linker flags for OclWrapper
    OPENCL_DIR=os.environ['OPENCL_DIR']
    OCL_LDFLAGS =  ['-L.','-L'+OPENCL_DIR+'/OpenCLIntegration']
else:
    OCL_LDFLAGS =  []

if OSX == 1:
# Assuming MacPorts
    INCLPATH = ['/opt/local/include','/opt/local/include/openmpi-gcc49/','/opt/local/lib/openmpi-gcc49/']
    LIBPATH = ['/opt/local/lib','/opt/local/lib/openmpi-gcc49/']
else:
# test for devtoolset-2 ... so better use a var $DEVTOOLSETROOT?
    if os.path.exists('/opt/rh/devtoolset-2'):
        INCLPATH = ['/opt/rh/devtoolset-2/root/usr/include' ]
        LIBPATH = '/opt/rh/devtoolset-2/root/usr/lib'
    else:
# reasonable default ...
        NETCDF = os.environ.get('NETCDF_DIR')
        INCLPATH = [NETCDF + '/include']
        LIBPATH  = [NETCDF + '/lib']
#MPICH = os.environ.get('MPICH')
#INCLPATH = [NETCDF + '/include', MPICH + '/include']
#LIBPATH = [NETCDF + '/lib', MPICH + '/lib']
#        INCLPATH = [NETCDF + '/include', '/usr/include']
#        LIBPATH  = [NETCDF + '/lib', '/usr/local/lib']
#        INCLPATH = ['/usr/local/include', '/usr/include' ]
#        LIBPATH = '/usr/local/lib'
#INCLPATH += ['../OpenCL','../OpenCL/Wrappers']

envF.Append(F95FLAGS=FFLAGS)
envF.Append(F95PATH=['.',INCLPATH])
envF.Append(LIBPATH=['.',LIBPATH])
if OSX != 1:
    envF.Append(LIBS=['m'])

mpi_ext=''
if with_mpi == '1':
    mpi_ext='_mpi'

ocl_ext=''
if with_ocl == '1':
    envF.Append(LIBS=['OclWrapperF','stdc++','OclWrapper'])
    ocl_ext='_ocl'
    if OSX==1:
            envF.Append(FRAMEWORKS=['OpenCL'])
    else:
            envF.Append(LIBS=['OpenCL'])

if csources:
    envF.Append(LIBS=['csubs'])

prog = envF.Program('les_main'+ocl_ext+mpi_ext,ffsources)


SyntaxError: Missing parentheses in call to 'print' (OclBuilder.py, line 28)

### Build and run script

`./src/build_and_run_WV_MPI_no_nesting.sh`

In [1]:
%%bash
#!/bin/sh  

rm ./les_main_mpi

procPerRow=$1
procPerCol=$2

scons -s -f SConstruct.mac v=0 gr_debug=0 wv_debug=0 mpi_new_wv=1 nested=0 ocl=0 mpi=1 procPerRow=${procPerRow} procPerCol=${procPerCol} $3

mpiexec-openmpi-gcc7 -np $((procPerRow*procPerCol)) ./les_main_mpi 


4 total processes failed to start


rm: ./les_main_mpi: No such file or directory

scons: warning: Ignoring missing SConscript 'SConstruct.mac'
File "/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scons-3.0.1-py2.7.egg/EGG-INFO/scripts/scons", line 201, in <module>
--------------------------------------------------------------------------
mpiexec-openmpi-gcc49 was unable to launch the specified application as it could not access
or execute an executable:

Executable: ./les_main_mpi
Node: h134

while attempting to start process rank 0.
--------------------------------------------------------------------------


### Helper scripts for CPP macro handling

In `./aux` there are two scripts to help with CPP macro handling, in particular for the case of running the OpenCL compiler (see [Generating and building the GPU code](#Generating-and-building-the-GPU-code) for full details on this process).
The first is `run_cpp.pl` which runs `cpp` with a list of defined macros. The script will look for a file `macros.h` in the directory where it is run (in our case `src`), if this file does not exist it will not run `cpp` and just copy the source files. It will store the output files in `../PostCPP`.

Additionally, the script will look for an optional file `macros_to_skip.h` in the directory where it is run (in our case `src`). Blocks of code guarded by these macros will be removed from the code before running `cpp`, and replaced by `continue` lines. This code is store in `../PostCPP/PrePostCPP`.
The removed lines are stored in a file `stash.pl` and can be restored using the script `restore_stashed_lines.pl`, which stores the code in `./PostGen`.

The typical use is to run `run_cpp.pl` before running the OpenCL compiler `Autoparallel-fortran`, and then run `restore_stashed_lines` on the sources generated by `Autoparallel-fortran`:

    [src]$ perl ../aux/run_cpp.pl
    [src]$ cd ../PostCPP
    [PostCPP]$ AutoParallel-Fortran-exe ./adam.f95 ./bondv1.f95 ./feedbf.f95 ./les.f95 ./press.f95 ./velFG.f95 ./velnw.f95 -out ../GeneratedCode/ -main ./main.f95 -v -plat GPU
    [PostCPP]$ cd ../GeneratedCode
    [GeneratedCode]$ perl ../aux/restore_stashed_lines.pl ../src/stash.pl 
    
And the final code is placed in `PostGen`.

    [GeneratedCode]$ cd PostGen  

For convenience there is also a script `pp_stash.pl` which will print out the Fortran code in the file `stash.pl`.

## Overview of CPP macros used in the LES source

### MPI related     

To enable MPI in the code:

    MPI
    
Dimensions of the Cartesian process grid

    PROC_PER_ROW
    PROC_PER_COL    
    
Changes to original MPI code before 15 July 2017. Mostly use of collective operations instead of individual send/receive pairs.

    MPI_NEW_WV
    
Used in:

    anime.f95
    aveflow.f95
    common_sn.f95
    communication_helper_mpi.f95
    communication_helper_real.f95
    ifdata.f95
    main.f95

`MPI_NEW_WV2` is only used in `ifdata.f95`, the code guarded by this macro is *not finished*.
    
### Debugging

To switch on debug output:

    GR_DEBUG
    WV_DEBUG

### Testing

* `WV_TEST` is used in `set.f95` to limit the simulation time for testing.
* `TEST_SMALL_DOMAIN` in `params_common_sn.f95` is *obsolete*, it was used to limit the domain for testing on GPU.
    
###   Nesting

* `NESTED_LES` enables nesting using stretched grids. It is used in:

        anime.f95
        bondFG.f95
        bondv1.f95
        boundp.f95
        boundsm.f95
        communication_helper_real.f95
        grid.f95
        main.f95
        params_common_sn.f95


* `NESTED_LES2` in `communication_helper_real.f95` is *obsolete*.
* `SAVE_NESTED_GRID_ONLY` in `anime.f95` and `communication_helper_real.f95` is used for code that saves only the inner  nest of the domain. This is *not finished*.
    
### Performance evaluation    

`TIMINGS` prints out timings for each subroutine in the main time loop.
    
### Feature control

The following replace the run-time Fortran conditions

    I_ANIME
    I_AVEFLOW
    I_IFDATA_OUT

The following guard code that is no longer used. I think this code is *obsolete* but it is not my code so I keep it.

    TIMSERIS_FIXED    
    OLD_CODE

### I/O control

* `USE_NETCDF_OUTPUT` results in generation of NetCDT output files. However, I think they are *broken* as I can't visualise them in either Paraview or Panoply.
* `VERBOSE` prints out intermediate results at run time.
* `NO_IO` is mainly used to remove I/O commands from the code for GPU code generation.
    
###  Code variants 

These are macros that should have no effect on the simulation except for performance.

* `NO_GLOBAL_SOR` is used to disable calculation of the SOR error over the full domain. We can do this because the error is not actually used as a criterion for convergence. This is an advantage for both MPI code and GPU code.
* `TWINNED_BUFFER` is an essential performance optimization for GPU: it replaces the red-black SOR algorithm with my twinned double buffering algorithm [ieeexplore.ieee.org/abstract/document/7237073/](ieeexplore.ieee.org/abstract/document/7237073/)
* `INLINE_BOUND_CALCS` is another GPU variant, the reason is because our OpenCL compiler can't handle calls to subroutines in kernel code.
* `WV_NEW`: This macro is used to guard a lot of changes aimed at reducing the memory footprint of the LES, see below.



## Reducing LES memory footprint

The main reason for reducing the LES memory footprint is to allow larger domains to be simulated on the GPU. However, this should also result in far less communication in the MPI version.

The main changes to reduce the footprint were to remove a number of intermediate arrays by replaycing them with inlined computations.

The changes to the code are guarded by the macro `WV_NEW`. The main changes are as follows:
* I moved the calculation of the mask for the buildings into the SOR algorithm. This eliminates the arrays `mask*` and `cn*`.
* I moved the calculations from `vel2` into `velFG` and scalarized them. This eliminates `diu*`, `cov*` and `nou*`.
* In `feedbf` I scalarized the calculations involving `fx`, `fy` and `fz`, eliminating these arrays.

In total this frees up half of the memory for the OpenCL case and two thirds for the MPI case.



## Generating the GPU version

The process to generate the GPU version consists of several steps and uses an additional compiler.


### Installing the `OclWrapper` library

This library provides a Fortran wrapper around the OpenCL C API. The library is [on GitHub](https://github.com/wimvanderbauwhede/OpenCLIntegration) and you need the Python build tool [scons](http://scons.org/) to build it. 

* Clone the repository:

        git clone https://github.com/wimvanderbauwhede/OpenCLIntegration

* Follow the instructions in the `README.md`, in particular you should put the content of `ocl_env.sh` in your `.bashrc` (Linux) or `.profile` (Mac) and adapt it for your system.

* You don't need to build the library, this is done later when the rest of the code is built.


### Building and installing the Fortran to GPU OpenCL compiler

This compiler generates OpenCL host code using the `OclWrapper` library and produces parallelised kernel code in Fortran.

The compiler is [on GitHub](https://github.com/wimvanderbauwhede/AutoParallel-Fortran) and you need the Haskell build tool [stack](https://www.haskellstack.org/) to build it. 


* Install [stack](https://docs.haskellstack.org/en/stable/README/)
* Clone the repository:

        git clone https://github.com/wimvanderbauwhede/AutoParallel-Fortran
    
* In the repository folder, run 

        stack install
        
* This will install the compiler executable `AutoParallel-Fortran-exe` in `$HOME/.local/bin/`. I will assume that this path is in your `$PATH`.


### Installing the `RefactorF4Acc` compiler

The compiler is [on GitHub](https://github.com/wimvanderbauwhede/RefactorF4Acc) and all you need is `Perl`, v5.16 or later. We use it here to translate the Fortran kernel code into OpenCL C code.

* Clone the repository:

        git clone https://github.com/wimvanderbauwhede/RefactorF4Acc            
           
* No further installation is necessary. Below, I have assumed that the local repository path is `$HOME/Git/RefactorF4Acc` but you can put it anywhere you like.

### Generating and building the GPU code

I have created a script that performs all actions described below, so all you need to do is run the following command:

    perl ./aux/generate_and_build_GPU_version.pl
       
in the `MPI-LES` directory. 

The `macros.h` and `macros_to_skip.h` files in `src` should be up-to-date, so I think there is no need to change anything there.

### What this script does (for information only)

The script takes the following (optional) arguments: 

        --dev: the target device: CPU or GPU. Default is $plat
        --nth: the number of threads per compute unit. Default is $nth  
        --nunits: the number of compute units (--nunits). Default is $nunits
        --stage: the stage of the conversion: refactor, autopar, convert, build.
                 
You can provide several stages separated with a comman. 
If not given, the script will attempt to run all stages in one go. 

The stages are:

    1. refactor: Refactor the F77 code into accelerator-ready F95 code. 
    The refactoring source-to-source compiler `rf4a` will only run if 
        - the source files have extension `.f`, `.f77`, `.F` or `.F77`, and 
        - the configuration file `rf4a.cfg` is present.
    2. autopar: Auto-parallelise the host code and generate the kernel in F95 syntax. This step requires the definition of macros used in the code, in two ways:
        - Macros to be expanded using the C preprocessor. You must define/undef these in the file `macros.h`. The script will warn if this file is not present.
        - Macros enclosing code to be skipped by the compiler. This is used  in particular because the compiler can't handle IO operations.
        You must define/undef these in the file `macros_to_skip.h`. The script will warn if this file is not present.
    3. convert: Convert the kernel to OpenCL. 
    The OpenCL kernel code uses two macros, the number of threads per compute unit `NTH` and the number of compute units `NUNITS`. 
    These can be defined using the --nth and --nunits flags or in the file `macros_kernel.h`.
    4. build: Build the OpenCL Fortran code. The code is built using an auto-generated SConstruct file. 
    You can of course modify this file and build the code manually. The script will not overwrite an existing `Sconstruct.auto` file.


#### GPU code generation and build script source 

The source code of the script is listed below FYI.


In [ ]:
%%perl
#!/usr/bin/env perl
use v5.10;
use strict;
use warnings;

# add the './aux' path to the list of include paths
BEGIN {
    push @INC, './aux';
};

use Cwd;
# A module to run the C preprocessor
use RunCpp qw( run_cpp );
# Mistery module
use RestoreStashedLines qw( restore_stashed_lines );
# This converts macro definitions from .h file to command line flag macro syntax
use MacroFileToCmdLine qw( macro_file_to_cmd_line_str );

use Data::Dumper;
use Getopt::Long;

=pod

# Name 

  generate_and_build_OpenCL_version.pl

# Description

The purpose of this script is to convert sequential Fortran 77 code to Fortran 95 code with parallelised OpenCL kernels which can be offloaded to a GPU or run multithreaded on the CPU. 

# Prerequisites

# Source code and directory structure requirements

The compilers used by the script do not work on all Fortran 77 or Fortran 95 code. However, it is usually possible with little effort to change the code and selected the correct compilation options to make it work.

## Source code requirements

# Usage
=cut

my $plat = 'GPU';
my $nth = 256;
my $nunits = 16;
my $vvv=0;

my $help_message =<<ENDH;

        $0 [-d, --dev CPU|GPU] [-s, --stage stage] [--nth #threads ] [--nunits #units]   

    The script takes the following (optional) arguments: 
        --dev: the target device: CPU or GPU. Default is $plat
        --nth: the number of threads per compute unit. Default is $nth  
        --nunits: the number of compute units (--nunits). Default is $nunits
        --stage: the stage of the conversion: refactor, autopar, convert, build.
                 
    You can provide several stages separated with a comman. 
    If not given, the script will attempt to run all stages in one go. 

    The stages are:

    1. refactor: Refactor the F77 code into accelerator-ready F95 code. 
    The refactoring source-to-source compiler `rf4a` will only run if 
        - the source files have extension `.f`, `.f77`, `.F` or `.F77`, and 
        - the configuration file `rf4a.cfg` is present.
    2. autopar: Auto-parallelise the host code and generate the kernel in F95 syntax. This step requires the definition of macros used in the code, in two ways:
        - Macros to be expanded using the C preprocessor. You must define/undef these in the file `macros.h`. The script will warn if this file is not present.
        - Macros enclosing code to be skipped by the compiler. This is used  in particular because the compiler can't handle IO operations.
        You must define/undef these in the file `macros_to_skip.h`. The script will warn if this file is not present.
    3. convert: Convert the kernel to OpenCL. 
    The OpenCL kernel code uses two macros, the number of threads per compute unit NTH and the number of compute units NUNITS. 
    These can be defined using the --nth and --nunits flags or in the file `macros_kernel.h`.
    4. build: Build the OpenCL Fortran code. The code is built using an auto-generated SConstruct file. 
    You can of course modify this file and build the code manually. The script will not overwrite an existing `Sconstruct.auto` file.
    
ENDH
my $help=0;

my $stages_str='refactor,autopar,convert,build';
my $use_separate_stash_step=0;
my $verbose;
GetOptions ('nth=i' => \$nth,                   
            'nunits=i' => \$nunits,
            'dev=s'   => \$plat,     # I know, not consistent.
            'stage=s' => \$stages_str,
            'stash' => \$use_separate_stash_step,
            'verbose'  => \$vvv,
            'help' => \$help
        ) or die("Error in command line arguments\n");

if ($help) { die $help_message; }        
my $wd = cwd();

my $VV=1;
my $vflag= $vvv ? '-b' : '';

my $main_src = 'main.f95';

# TODO: These should be extracted from the source code using rf4a. It would be best to save these to a file when running the refactoring

my @kernel_sources=qw(
adam.f95
bondv1.f95
feedbf.f95
les.f95
press.f95
velFG.f95
velnw.f95
);

# TODO: These should be extracted from the source code using rf4a

my @iowrite_subs=qw(
'anime'
);

# TODO: These should be extracted from the source code using rf4a, because they are the source used for building the code minus the kernel sources.
# However, that would probably result in many unused files being copied. So we could just copy the src folder 
#bondv1.f95
my @orig_sources=qw(
anime.f95
aveflow.f95
bondFG.f95
boundp.f95
boundsm.f95
common_sn.f95
feedbfm.f95
grid.f95
ifdata.f95
init.f95
params_common_sn.f95
set.f95
timdata.f95
timseris.f95
macros.h
macros_to_skip.h
);

#

my $iowrite_subs_str = join(' ',@iowrite_subs);
my $kernel_sources_str = join(' ',map {"./$_" } @kernel_sources);

my @sub_names = map {s/\.f95$//;$_ } @kernel_sources;

# This 30 character limit was picked ad-hoc by Gavin
my $superkernel_name = substr(join('_',@sub_names),0,30);
if (length($superkernel_name)==30) {
    $superkernel_name .= "_etc_superkernel"
} else {
    $superkernel_name .= "_superkernel"
}

my $TRUST_THE_COMPILER = 1 - $use_separate_stash_step;

my $skip_step_0 = 1;
my $skip_step_1 = 1;
my $skip_step_2 = 1;
my $skip_step_3 = 1;

my %stages = map { $_ => $_} split(/\s*,\s*/,$stages_str);

if (exists $stages{refactor}) {
    $skip_step_0 = 0;
}
if (exists $stages{autopar}) {
    $skip_step_1 = 0;
}
if (exists $stages{convert}) {
    $skip_step_2 = 0;
}
if (exists $stages{build}) {
    $skip_step_3 = 0;
}

my $gen_dir = 'GeneratedCode';
if ($TRUST_THE_COMPILER==1) {
    $gen_dir = 'GeneratedCodeV2';
}

# The compiler fails if this directory does not exists
if (not -d $gen_dir) {
    mkdir $gen_dir;
}


# this is LES-specific
chdir $gen_dir;
if (not -d 'data') {
    system('cp -r ../data .');
}
if (not -d 'GIS') {
    system('cp -r ../GIS .');
}
chdir $wd;
 
my $refactored=0;

# Stage 1. Check if the code needs to be refactored from F77 to F95
# If so, refactor it; if not, say why not.
if (not $skip_step_0) {
    chdir 'src';
    my @f77_sources = glob('*.f77 *.F77 *.f *.F');
    my $has_F77_code = scalar @f77_sources > 0;
    my $has_rf4a_cfg = -e './rf4a.cfg';
    if ( $has_rf4a_cfg and $has_F77_code) {
        $refactored=1;

        say "Refactoring F77 code into accelerator-ready F95 code";
        say($ENV{HOME}.'/Git/RefactorF4Acc/bin/'.'refactorF4acc.pl -c ./rf4a.cfg '.$vflag); 
        system($ENV{HOME}.'/Git/RefactorF4Acc/bin/'.'refactorF4acc.pl -c ./rf4a.cfg '.$vflag); 
 
    } else {
        say "Refactoring step stage skipped because already done:\n";
        if (!$has_rf4a_cfg) {
        say "\t- No rf4a.cfg file"; 
        }
        if(!$has_F77_code) {
            say "\t-No F77 source files";
        }
        say '';
    }
}
    my $src_dir = $refactored ? 'RefactoredSources' : 'src';

# Stage 2. Run the auto-parallelizing GPU compiler `AutoParallel-Fortran-exe`. The output is stored in `GeneratedCodeV2`
if (not $skip_step_1) {
    if ($TRUST_THE_COMPILER) {
        chdir $src_dir;
        ##
        say'*NOTE 2018-03-07* 
        The `AutoParallel-Fortran` compiler has built-in handling of macros via the -D and -X flags. 
        This generates the same code as when using the `run_cpp.pl` and `restore_stashed_lines.pl` scripts. 
        ' if 0;
        
        (my $defined_macros_str, my $undef_macros_str) = macro_file_to_cmd_line_str( './macros.h','-D');
        (my $macros_to_skip_str, my $empty_str) = macro_file_to_cmd_line_str('./macros_to_skip.h','-X');
        
        say("AutoParallel-Fortran-exe $kernel_sources_str -out ../$gen_dir/ -iowrite $iowrite_subs_str -main ./$main_src -plat $plat  $defined_macros_str $macros_to_skip_str $vflag" );
        #system('which AutoParallel-Fortran-exe');die;
        system("AutoParallel-Fortran-exe $kernel_sources_str -out ../$gen_dir/ -iowrite $iowrite_subs_str -main ./$main_src  -plat $plat  $defined_macros_str $macros_to_skip_str $vflag" );    
        
    } else {    
    
        say '* First, in `'.$src_dir.'`, run CPP on the code using the macros in `macros.h` and stash lines guarded with macros from `macros_to_skip.h`. This generates the file `stash.pl`' if $VV;
        
        chdir $src_dir;
        
        run_cpp();
          
        ##
        say '* Then, in `PostCPP`, run the OpenCL compiler `AutoParallel-Fortran-exe`. This will take a while and produce a lot of output, which you can ignore.' if $VV;
        
        chdir $wd;
        if (not -d 'PostCPP') {
            mkdir 'PostCPP';
        }
         
        chdir 'PostCPP';
        say("AutoParallel-Fortran-exe $kernel_sources_str -out ../$gen_dir/ -iowrite $iowrite_subs_str -main ./$main_src $vflag -plat $plat");
        system("AutoParallel-Fortran-exe $kernel_sources_str -out ../$gen_dir/ -iowrite $iowrite_subs_str -main ./$main_src $vflag -plat $plat");
        
        ##
        say "* In '$gen_dir', we restore code segments that were stashed in the previous step" if $VV;
        chdir $wd;
        chdir $gen_dir;
        
        restore_stashed_lines("$wd/$src_dir/stash.pl"); 
        system('cp ./PostGen/* .');
    
    }
}
# Stage 3. Copy non-modified source files and scripts and config files needed to build the OpenCL kernel, and generate the OpenCL kernel
if (not $skip_step_2) { 
    
    ##
    say "* In `$gen_dir`, we copy the non-modified source files into the current folder, as well as some scripts and config files needed to build the OpenCL kernel." if $VV;
    chdir $wd;
    chdir $gen_dir;
    
    
    
    my $ref_dir = $TRUST_THE_COMPILER ? "$wd/$src_dir" : "$wd/PostCPP";
    for my $src (@orig_sources) {
       system("cp $ref_dir/$src ."); 
    }

    ## TODO rf4a_to_C.cfg should be generated
    my $rf4a_to_C_cfg = <<ENDCFG;
MODULE = module_${superkernel_name}
MODULE_SRC = module_${superkernel_name}.f95
TOP = ${superkernel_name}
KERNEL = ${superkernel_name}
PREFIX = .
SRCDIRS = .
NEWSRCPATH = ./Temp
EXCL_SRCS = (module_${superkernel_name}_init|_host|\\.[^f])
EXCL_DIRS = ./PostCPP,./Temp
MACRO_SRC = macros_kernel.h

ENDCFG
    
    open my $CFG, '>', 'rf4a_to_C.cfg';
    print $CFG $rf4a_to_C_cfg;
    close $CFG;
    
    my @sources2=qw(
    macros_kernel.h
    array_index_f2c1d.h
    );
    
    my $ref_dir_2 = "$wd/aux";
    for my $src (@sources2) {
        system("cp $ref_dir_2/$src . ");
    }

    ##
    say '* Then we generate the actual OpenCL kernel code using `RefactorF4Acc`' if $VV;
    chdir $wd;
    chdir $gen_dir;
    
    my $macros_kernel_src = './macros_kernel.h';
    
    if (not -e $macros_kernel_src ) {
        say "No `macros_kernel.h` file for the macros NTH and NUNITS";
        say "Creating one with NTH=$nth and NUNITS=$nunits";
        open my $MKS, '>', 'macros_kernel.h';
        say $MKS "#define NTH $nth";
        say $MKS "#define NUNITS$nunits";
        close $MKS;

    }
    say($ENV{HOME}.'/Git/RefactorF4Acc/bin/'.'refactorF4acc.pl '.$vflag.' -P translate_to_OpenCL -c rf4a_to_C.cfg '.$superkernel_name); 
    system($ENV{HOME}.'/Git/RefactorF4Acc/bin/'.'refactorF4acc.pl '.$vflag.' -P translate_to_OpenCL -c rf4a_to_C.cfg '.$superkernel_name);
    system("cp  module_$superkernel_name.cl module_${superkernel_name}_ORIG.cl");

    # Unused, for debugging
    #open my $MK, '<', $macros_kernel_src or die $!;
    #my @ls=<$MK>;
    #close $MK;
    #my $macros_str=join(" ",map {
    #    $_=~s/\n//;
    #    s/^\s*//;
    #    s/\s*$//;
    #    s/.define\s*/-D/;
    #    s/.undef\s*/-U/;
    #    s/\s+/=/;
    #    $_
    #} @ls);
    #say("cpp $macros_str -I. -P module_$superkernel_name.cl > module_${superkernel_name}_after_CPP_for_debugging.cl");
    #system("cpp $macros_str -I. -P module_$superkernel_name.cl > module_${superkernel_name}_after_CPP_for_debugging.cl");

}

# Stage 4. Build the host code for the OpenCL kernel
if (not $skip_step_3) {
##
    chdir $wd;
    chdir $gen_dir;
    
    ## SConstruct.auto is generated
    create_sconstruct($main_src, \@kernel_sources, \@orig_sources, $superkernel_name);
    
    say  "Now we can build the OpenCL Fortran host code, setting the number of threads and compute units depending on the GPU";
    say 'Note that the Scons build runs cpp on the kernel for the macros NTH, NUNITS and BARRIER_OK';
    say 'Normally these are set via the `nth`, `nunits` and `dev` flags on the scons command line';
    say 'But you can also put them in `macros_kernel.h`';
    
    say("scons -f SConstruct.auto -s mcm=m dev=$plat nth=$nth nunits=$nunits");
    system("scons -f SConstruct.auto -s mcm=m dev=$plat nth=$nth nunits=$nunits");
}

# ----- Helper functions -----
# 
sub create_sconstruct { (my $main_src, my $kernel_sources, my $orig_sources, my $superkernel_name)=@_;

    my @host_srcs = map {
        my $name = strip_ext($_);
        my $host_name = $name.'_host';
        my $host_src = "'$host_name.f95'";
        $host_src
    } ($main_src, @{ $kernel_sources } );

    my $host_srcs_str = join(',',@host_srcs);
    my @q_orig_sources = map { "'./$_'" } @{ $orig_sources };
    my $orig_srcs_str = join(',',@q_orig_sources);

    my $module_init_str = "'./module_${superkernel_name}_init.f95'";
    my $kernel_src_cl_str = "'module_${superkernel_name}.cl'";

    if (not -e "SConstruct.auto" ) {
        open my $SCONS_TEMPL,'<',"$wd/aux/SConstruct.templ";    
        open my $SCONS,'>', "SConstruct.auto";
        while (my $line= <$SCONS_TEMPL> ) {
            $line=~/__HOST_SRCS__/ && do {
                $line=~s/__HOST_SRCS__/$host_srcs_str/;            
            }; 
            $line=~/__MODULE_INIT__/ && do {
                $line=~s/__MODULE_INIT__/$module_init_str/;
            }; 
            $line=~/__KERNEL_SRC_CL__/ && do {
                $line=~s/__KERNEL_SRC_CL__/$kernel_src_cl_str/;
            };
            $line=~/__ORIG_SOURCES__/ && do {
                $line=~s/__ORIG_SOURCES__/$orig_srcs_str/;
            }; 
            print $SCONS $line;
        }
        close $SCONS;
        close $SCONS_TEMPL;
    } else {
        say "SConstruct.auto already exists, not overwriting. Delete or rename the file and run the build stage again.";
    }

}

sub strip_ext { (my $fn)=@_;
    $fn=~s/\.\w+$//;
    return $fn;
}
